# ibm_db.foreign_keys()

## Purpose:

Retrieve information about the columns that participate in foreign keys that reference a table.

## Syntax:

`IBM_DBStatement ibm_db.foreign_keys( IBM_DBConnection `*`connection,`*` string `*`qualifierName,`*` string `*`schemaName,`*` string `*`tableName`*` )`

## Parameters:

* __*connection*__&nbsp; &nbsp; &nbsp; &nbsp; : A valid Db2 server or database connection. 
* __qualifierName__&nbsp; &nbsp; : A valid qualifier name for Db2 databases on OS/390 or z/OS servers; the value `None` or an empty string (`''`) for Db2 databases on other operating systems. 
* __schemaName__&nbsp; &nbsp; &nbsp;: The name of the schema that contains the table(s) that foreign key information is to be obtained for. To use the schema associated with the current connection, provide the value `None`.
* __tableName__&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; : The name of the table(s) that foreign key information is to be obtained for.  

## Return Values:

* If __successful__, an IBM_DBStatement with a result set that contains the following information:<p>

    * `PK_NAME`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; : The name of the primary key.
    * `PKTABLE_CAT` &nbsp; &nbsp; &nbsp; &nbsp;: The name of the catalog associated with the schema and table that contains the primary key; Db2 does not use catalogs so this field will always contain the value `None`. *(Db2 databases on OS/390 or z/OS servers can return information in this field.)*
    * `PKTABLE_SCHEM` &nbsp; &nbsp;: The name of the schema for the table that contains the primary key.
    * `PKTABLE_NAME` &nbsp; &nbsp; &nbsp;: The name of the table that contains the primary key.
    * `PKCOLUMN_NAME`&nbsp; &nbsp; : The name of a primary key column.
    * `FK_NAME`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; : The name of the foreign key.
    * `FKTABLE_CAT` &nbsp; &nbsp; &nbsp; &nbsp;: The name of the catalog associated with the schema and table that contains the foreign key; Db2 does not use catalogs so this field will always contain the value `None`. *(Db2 databases on OS/390 or z/OS servers can return information in this field.)*
    * `FKTABLE_SCHEM` &nbsp; &nbsp;: The name of the schema for the table that contains the foreign key.
    * `FKTABLE_NAME` &nbsp; &nbsp; &nbsp;: The name of the table that contains the foreign key.
    * `FKCOLUMN_NAME`&nbsp; &nbsp; : The name of a foreign key column.    
    * `KEY_SEQ`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; : The __FKCOLUMN_NAME__ column's ordinal position in the foreign key, starting from 1.
    * `UPDATE_RULE`&nbsp; &nbsp; &nbsp; &nbsp; : The action to be applied to the foreign key when an `UPDATE` operation is performed.
    * `DELETE_RULE`&nbsp; &nbsp; &nbsp; &nbsp; : The action to be applied to the foreign key when a `UDELETE` operation is performed.
    * `DEFERRABILITY`&nbsp; &nbsp; : Indication of whether the foreign key is deferred initially and applied later, applied immediately, or cannot be deferred.<p>
    
* If __unsuccessful__, the value `False`.

## Description:

The __ibm_db.foreign_keys__ API is used to retrieve information about the columns that participate in foreign keys that reference a table.<p>
    
The information returned by this API is placed in a result data set, which can be processed using the same APIs that are used to process result data sets that are generated by SQL queries. That is, a single row can be retrieved and stored in a tuple or dictionary using the  __ibm_db.fetch_tuple()__ (tuple), __ibm_db.fetch_assoc()__ (dictionary), or __ibm_db.fetch_both()__ (tuple *and* dictionary) APIs. Alternately, the __ibm_db.fetch_row()__ API can be used to move the result set pointer to each row in the result set produced and the __ibm_db.result()__ API can be used to fetch a column from the current row.

## Example:

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-foreign_keys.py                                                            #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.foreign_keys() API.  #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.tables()                                                              #
#                 ibm_db.fetch_assoc()                                                         #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The Db2ConnectionMgr Class Definition, Attributes, And Methods That Have Been Defined #
# In The File Named "ibm_db_tools.py"; This Class Contains The Programming Logic Needed To     #
# Establish And Terminate A Connection To A Db2 Server Or Database                             #
#----------------------------------------------------------------------------------------------#
from ibm_db_tools import Db2ConnectionMgr

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
dbName = "SAMPLE"
userID = "db2inst1"
passWord = "Passw0rd"
dbConnection = None
schemaName = userID.upper()
tableName = "PROJECT"
resultSet = False
dataRecord = False
foreignKeyRules = {0 : "CASCADE", 1 : "RESTRICT", 2 : "SET_NULL", 3 : "NO_ACTION", 
                   4 : "SET_DEFAULT"}
deferability = {5 : "INITIALLY_DEFERRED", 6 : "INITIALLY_IMMEDIATE", 
                7 : "NOT_DEFERRABLE"}

# Create An Instance Of The Db2ConnectionMgr Class And Use It To Connect To A Db2 Database
conn = Db2ConnectionMgr('DB', dbName, '', '', userID, passWord)
conn.openConnection()
if conn.returnCode is True:
    dbConnection = conn.connectionID
else:
    conn.closeConnection()
    exit(-1)

# Attempt To Retrieve Information About All Of The Tables In The Current User's Schema
print("Obtaining information about tables in the " + schemaName + " schema ... ", end="")
try:
    resultSet = ibm_db.tables(dbConnection, None, schemaName, None, 'TABLE')
except Exception:
    pass

# If The Information Desired Could Not Be Retrieved, Display An Error Message And Exit
if resultSet is False:
    print("\nERROR: Unable to obtain the information desired\n.")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# As Long As There Are Records (That Were Produced By The ibm_db.tables API), ... 
tableList = []
noData = False
while noData is False:

    # Retrieve A Record And Store It In A Python Dictionary
    try:
        dataRecord = ibm_db.fetch_assoc(resultSet)
    except:
        pass

    # If The Data Could Not Be Retrieved Or There Was No Data To Retrieve, Set The
    # "No Data" Flag And Exit The Loop  
    if dataRecord is False:
        noData = True

    # Otherwise, Store The Table Name Retrieved In A Python List (This List Will Be Used
    # Later To Skip Records Associated With Aliases)
    else:
        tableList.append(dataRecord['TABLE_NAME'])

# Attempt To Retrieve Information About All Of The Foreign Keys That Have Been Defined For 
# The Table Specified
print("Obtaining information about foreign keys created for ", end="")
print("the " + schemaName + "." + tableName + " table ... ", end="")
resultSet = False
try:
    resultSet = ibm_db.foreign_keys(dbConnection, None, schemaName, tableName)
except Exception:
    pass

# If The Information Desired Could Not Be Retrieved, Display An Error Message And Exit
if resultSet is False:
    print("\nERROR: Unable to obtain the information desired\n.")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# As Long As There Are Records (That Were Produced By The ibm_db.foreign_keys API), ...
noData = False
loopCounter = 1
while noData is False:

    # Retrieve A Record And Store It In A Python Dictionary
    try:
        dataRecord = ibm_db.fetch_assoc(resultSet)
    except:
        pass

    # If The Data Could Not Be Retrieved Or If There Was No Data To Retrieve, Set The
    # "No Data" Flag And Exit The Loop  
    if dataRecord is False:
        noData = True

    # Otherwise, Display The Data Retrieved (Skip Information For Aliases)
    else:

        # If The Current Data Record Is For A Table Alias, Ignore It
        if not dataRecord['FKTABLE_NAME'] in tableList:
            continue

        # Display Record Header Information
        print("Foreign key " + str(loopCounter) + " details:")
        print("______________________________________________")

        # Display The Information Stored In The Data Record Retrieved
        print("Primary key name    : {}" .format(dataRecord['PK_NAME']))
        print("  Table schema      : {}" .format(dataRecord['PKTABLE_SCHEM']))
        print("  Table name        : {}" .format(dataRecord['PKTABLE_NAME']))
        print("  Column name       : {}" .format(dataRecord['PKCOLUMN_NAME']))
        print("Foreign key name    : {}" .format(dataRecord['FK_NAME']))
        print("  Table schema      : {}" .format(dataRecord['FKTABLE_SCHEM']))
        print("  Table name        : {}" .format(dataRecord['FKTABLE_NAME']))
        print("  Column name       : {}" .format(dataRecord['FKCOLUMN_NAME']))
        print("  Column number     : {}" .format(dataRecord['KEY_SEQ']))
        print("  UPDATE rule used  : ", end="")
        print(foreignKeyRules.get(dataRecord['UPDATE_RULE']))
        print("  DELETE rule used  : ", end="")
        print(foreignKeyRules.get(dataRecord['DELETE_RULE']))
        print("  Deferability      : ", end="")
        print(deferability.get(dataRecord['DEFERRABILITY']))

        # Increment The loopCounter Variable And Print A Blank Line To Separate Foreign Key
        # Records From Each Other
        loopCounter += 1
        print()

# Close The Database Connection That Was Opened Earlier
conn.closeConnection()

# Return Control To The Operating System
exit()


Connecting to the SAMPLE database ... Done!

Obtaining information about tables in the DB2INST1 schema ... Done!

Obtaining information about foreign keys created for the DB2INST1.PROJECT table ... Done!

Foreign key 1 details:
______________________________________________
Primary key name    : PK_PROJECT
  Table schema      : DB2INST1
  Table name        : PROJECT
  Column name       : PROJNO
Foreign key name    : RPAP
  Table schema      : DB2INST1
  Table name        : PROJACT
  Column name       : PROJNO
  Column number     : 1
  UPDATE rule used  : NO_ACTION
  DELETE rule used  : RESTRICT
  Deferability      : NOT_DEFERRABLE

Foreign key 2 details:
______________________________________________
Primary key name    : PK_PROJECT
  Table schema      : DB2INST1
  Table name        : PROJECT
  Column name       : PROJNO
Foreign key name    : RPP
  Table schema      : DB2INST1
  Table name        : PROJECT
  Column name       : MAJPROJ
  Column number     : 1
  UPDATE rule used  : NO_